# <center> Анализ объявлений и построение прогнозной модели посещаемости 
<center> Автор: Михаил Усков

<center> На основе это проекта написана [статья](https://habrahabr.ru/post/280500/) на Хабре.

Для выполнения проекта была поставлена задача построить модель, которая могла бы по тексту объявления в интернете спрогнозировать посещаемость данного объявления. Для получения данных был разработан краулер, который скачивал объявления с сайта и формировал итоговый csv-файл. Тематика объявлений была одинакова для всех - передача собак из приютов и от частных лиц в добрые руки. 

## Загрузка библиотек

In [ ]:
%matplotlib inline

import datetime

import warnings
warnings.filterwarnings('ignore')

import seaborn

# кириллица на графиках
from matplotlib import rc
font = {'family': 'Verdana',
        'weight': 'normal'}
rc('font', **font)

import matplotlib.pyplot as plt



from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', True)

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.learning_curve import validation_curve,learning_curve

from sklearn.base import TransformerMixin

from sklearn.feature_selection import SelectKBest, chi2



from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
import Stemmer

## Описание данных

В исходных данных содержится название, описание и общее число посещений объявления с начала публикации.

### Список переменных

1. description - полный текст объявления
+ identificator - номер объявления на сайте
+ num_counts - число посещений объявления с начала его размещения
+ price - цена, за которую предлагается купить животное. обычно, волонтеры ставят 100р. или вовсе не указывают цену.
+ start_date - дата, когда объявление было размещено
+ title - название объявления, как оно выглядит на первой странице.

In [ ]:
dataset = pd.read_csv('../../data/billdoard_dataset.csv', header = 0)

In [ ]:
dataset.head()

### Описательные статистики

In [ ]:
# посещаемость
dataset['num_counts'].hist()
plt.title('Number of views')
plt.show()

# логарифм посещаемости
dataset['num_counts'].apply(np.log).hist()
plt.title('Log of Number of views')
plt.show()

In [ ]:
# цена
print(pd.value_counts(dataset['price']).head(20))

Число слов в объявлении:

In [ ]:
num_of_words = dataset['description'].apply(lambda x: len(x.split(' ')))
plt.hist(num_of_words)
plt.title('Distribution of the number of words')

Диаграммы рассеяния будут построены после предобработки

## Предобработка

1. Признаками будут являться отдельные слова или биграммы, поэтому их необходимо нормализовать, чтобы одно и то же слово, но в разных формах не считалось как два разных. Эта операция называется стемминг, что согласно вики значит процесс нахождения основы слова для заданного исходного слова. 
2. Поле "date" содержит дату в форме строки, поэтому ее нужно перевести в правильный формат
3. В качестве признака берется поле "description", поэтому текст необходимо перевести в представление "bag of words" и применить tf-idf. При этом из текста убираются предлоги, вспомогательные частицы и т.д. Эти слова называются stopwords.
4. Поскольку каждое объявление имеет свою дату публикации, то целевая переменная здесь представляет не просто число просмотров, а число просмотров, разделенное на число дней с момента подачи до скачивания их моим краулером. Т.е. можно считать, что это грубая оценка числа посетителей в сутки.
5. После нескольких неудачных попыток восстановить регрессию между document-term matrix и средним числом посетителей, было принято решение разбить целевую переменную на интервалы(квартили) и рассматривать задачу классификации (отсюда и tf-idf). Т.е. на выходе модель будет прогнозировать интервал, где содержится средняя посещаемость для данного объявления. Преобразование в квартили производилось только на обучающей выборке, поэтоу необходимо написать функцию, которая преобразовает и тестовую. Преобразовывать всю выборку целиком нельзя, посольку тогда тестовые данные будут косвенно участвовать в обучении.
6. Поле 'price' представляет собой цену за животное. Большие цены являются индикатором продажи породистого животного, нас же интересует некоммерческая деятельность, поэтому оставляем только те записи, для которых price < 500р.
7. Разбиение на train\test. Причем на train будет проводится обучение и подбор параметров по сетке на кросс-валидации, а на test будет проверяться финальное качество. Основная метрика - accuracy

In [ ]:
# стемминг
stop = stopwords.words('russian')
stemobject = Stemmer.Stemmer('russian')#RussianStemmer()
def stemmer(x):
    stem_function = stemobject.stemWord
    out = [stem_function(word) for word in x.split(' ')]
    return ' '.join(out)

# преобразование тестовых данных в квартили
def y_transform(x):
    
    if isinstance(x,str):
        return x
    
    categories = y_train.unique().categories.get_values()
    
    first = categories[0]
    first = first.replace('[','').replace(']','').split(',')
    first = [float(i) for i in first]

    last = categories[-1]
    last = last.replace('(','').replace(']','').split(',')
    last = [float(i) for i in last]
    
    if (x < first[0]):
        return categories[0] 
        
    if (x >= first[0]) & (x <= first[1]):
        return categories[0]
    
    if (x >= last[0]) & (x <= last[1]):
        return categories[-1] 
    
    if (x >= last[1]):
        return categories[-1]  
        
    for cat in categories[1:-1]:
        c = cat.replace('(','').replace(']','').split(',')
        c = [float(i) for i in c]
        if (x > c[0]) & (x <= c[1]):
            return cat
        
# обработка даты
month = ['января','февраля','марта','апреля','мая','июня','июля','августа','сентября','октября','ноября','декабря']
order = {M: i+1 for i,M in enumerate(month)}

def date_helper(x):
    now = datetime.datetime(2016,3,18)
    x = x.split(' ')
    d = x[0]
    m = order[x[1]]
    y = x[2]
    
    dateobject =  datetime.datetime(int(y),int(m),int(d))
    delta = now-dateobject
    return delta.days

In [ ]:
# идентификатор не несет полезной нагрузки
dataset.drop(['identificator'], axis = 1, inplace = True)

In [ ]:
dataset['num_days'] = dataset['start_date'].apply(date_helper)
dataset.drop(['start_date'], axis = 1, inplace = True)

Создание новой целевой переменной как отношения общего числа посещений к числу дней, прошедших с момента публикации до момента попадания объявления в выборку. В знаменателе добавляет единичка, чтобы избежать деления на нуль.

In [ ]:
dataset['mean_count'] = dataset['num_counts'] / (dataset['num_days']+1)
dataset.drop(['num_counts'], axis = 1, inplace = True)
dataset.drop(['num_days'], axis = 1, inplace = True)

In [ ]:
dataset.describe()

Ниже на графике видно, что чем меньше слов в объявлении, тем больше просмотров, если оценивать грубо. Предполагаю, что это справедливо только для объявлений данной конкретной тематики, поскольку чем больше слов, тем скорее всего сложнее история у животного.

In [ ]:
plt.scatter(num_of_words,dataset['mean_count'])
plt.xlabel('Number of words')
plt.ylabel('Mean Number of visits')

Рассмотрим те объявления, которые составляют топ-20 по среднему числу посещений:

In [ ]:
sorted_df = dataset.sort(['mean_count'], ascending=False)
for i in (sorted_df.index.tolist()[:10]):
    print('==================')
    print('\nНазвание: ')
    print('----------')
    print(sorted_df.loc[i,'title'])
    
    print('\nОписание: ')
    print('----------')
    print(sorted_df.loc[i,'description'])

    
    print('\nСреднее число просмотров: ',sorted_df.loc[i,'mean_count'])
    print('--------------------------')
    

Видно, что наибольшей популярностью пользуются попродистые собаки. Теперь рассмотрим наименее популярные объявления:

In [ ]:
sorted_df = dataset.sort(['mean_count'], ascending=True)
for i in (sorted_df.index.tolist()[:10]):
    print('==================')
    print('\nНазвание: ')
    print('----------')
    print(sorted_df.loc[i,'title'])
    
    print('\nОписание: ')
    print('----------')
    print(sorted_df.loc[i,'description'])

    
    print('\nСреднее число просмотров: ',sorted_df.loc[i,'mean_count'])
    print('--------------------------')

К сожалению, животные из приютов пользуются меньшей популярностью.

In [ ]:
plt.hist(np.log(dataset['mean_count']))

In [ ]:
# если цена указанна как 'Неуказана', 'Договорная' то она заменяется на 100р.
dataset['price'] = dataset['price'].apply(lambda x: 100 if x in ['Неуказана', 'Договорная'] else int(x))
dataset = dataset[dataset['price'] <= 500]
dataset.drop(['price'], axis = 1, inplace = True)
dataset.shape

Стемминг можно применить ко всему набору сразу, поскольку для работы он использует только данное конкретное слово

In [ ]:
dataset['description'] = dataset['description'].apply(stemmer) 

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(dataset['description'],dataset['mean_count'], test_size=0.33)

Разбиение целевой переменной на 5 квартилей. Т.е. получили задачу многоклассовой классификации из 5 классов

In [ ]:
y_train = pd.qcut(y_train, q= 5)

Рассмотрим baseline - qcut разбил выборку равномерно - по 454 объекта в каждом квартиле. Если всем объектам приписать какой-нибудь один класс, то качество будет 20%.

In [ ]:
print(pd.value_counts(y_train))
print(pd.value_counts(y_train))
print('Baseline accuracy: ',454/y_train.shape[0])

Сетка для поиска гиперпараметров.Поскольку интересно знать, какие слова влияют на посещаеомсть, было решенно не использовать LSA

In [ ]:
parameters = {
    'vect__max_df': (0.5, 0.75,1),
    #'vect__max_features': (10000, 50000),
    #'vect__ngram_range': ((1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True,False),
    #'tfidf__norm': ('l2'),
    #'svd__n_components':(50,100,1000),
    "clf__max_depth": [3, None],
    "clf__max_features": [1, 3, 10],
    "clf__min_samples_split": [1, 3, 10],
    "clf__min_samples_leaf": [1, 3, 10],
    "clf__bootstrap": [True, False]
}

total_comb = 1
for i in parameters.values():
    total_comb *= len(i)
print('Всего комбинаций: ', total_comb)

В качестве классификатора - случайный лес.

In [ ]:
pipe = Pipeline([('vect', CountVectorizer(stop_words=stop, ngram_range=(1, 2), max_features=10000)),
                 ('tfidf', TfidfTransformer(use_idf=True,norm='l2')),
                 #('svd', TruncatedSVD(algorithm='randomized')),
                 ('clf', RandomForestClassifier(n_estimators=500, verbose = False))]) 


grid_search = GridSearchCV(pipe, parameters, verbose=True, n_jobs = -1, cv = 5)
grid_search.fit(X_train, y_train)

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
# Преобразование тестовой выборки в квартили
y_test = y_test.apply(y_transform)

In [ ]:
print(classification_report(y_test,grid_search.predict(X_test)))

In [ ]:
print(pd.crosstab(y_test,grid_search.predict(X_test)))

Топ-10 слов, которые сильнее всего влияют на классификацию

In [ ]:
feature_weights = {'name': grid_search.best_estimator_.named_steps['vect'].get_feature_names(),
                  'weight': grid_search.best_estimator_.named_steps['clf'].feature_importances_}

feature_weights = pd.DataFrame(data=feature_weights['weight'], index = feature_weights['name'], columns = ['weight'])
feature_weights.sort(['weight'], ascending=False, inplace=True)
feature_weights.head(10)

In [ ]:
fig, ax = plt.subplots()
feature_weights.head(50).plot(ax=ax,kind = 'bar')
fig.autofmt_xdate()

## Диаграммы рассеяния(scatterplots)

Вообще, этот этап должен быть в начале исследования, но поскольку у нас текст и сильно разреженная матрица объекты-признаки, то имеет смысл построить графики только для топ-7, в соответствии с информацией о важности признаков, полученной от случайного леса.

In [ ]:
from sklearn.preprocessing import LabelEncoder

dataset_tfidf = pd.DataFrame(grid_search.best_estimator_.named_steps['vect'].transform(X_train).toarray())
dataset_tfidf.columns = grid_search.best_estimator_.named_steps['vect'].get_feature_names()
dataset_tfidf = dataset_tfidf[feature_weights.index.tolist()[:7]]

# поскольку у нас дискретные отсчеты, немного "размоем" значения, чтобы точки не ложились одна на другую:
for k in dataset_tfidf.keys():
    dataset_tfidf[k] = dataset_tfidf[k].apply(lambda x: int(x)+np.random.normal(0,0.08))

dataset_tfidf['target'] =(y_train)

seaborn.pairplot(dataset_tfidf, hue = 'target')


Видно, что желтые точки(интервал с максимальным числом просмотров) хорошо различаются от остальных

## Кривые обучения

In [ ]:
def plot_with_std(x, data, **kwargs):
        mu, std = data.mean(1), data.std(1)
        lines = plt.plot(x, mu, '-', **kwargs)
        plt.fill_between(x, mu - std, mu + std, edgecolor='none',
                         facecolor=lines[0].get_color(), alpha=0.2)
        
def plot_learning_curve(clf, X, y, scoring, cv=5):
 
    train_sizes = np.linspace(0.05, 1, 20)
    n_train, val_train, val_test = learning_curve(clf,
                                                  X, y, train_sizes, cv=cv,
                                                  scoring=scoring, n_jobs = -1)
    plot_with_std(n_train, val_train, label='training scores', c='green')
    plot_with_std(n_train, val_test, label='validation scores', c='red')
    plt.xlabel('Training Set Size'); plt.ylabel(scoring)
    plt.legend()

In [ ]:
plot_learning_curve(grid_search.best_estimator_,X_train, y_train, scoring='f1_weighted', cv=10)

**Примечание:** иногда при разных запусках на валидационной кривой можно увидеть, что классификатор всегда отлично описывает обучающую выборку, но зазор между train и cv кривыми довльно большой. Скорее всего это индикатор переобучения и вот откуда оно взялось: в качестве классификатора передается pipeline, **гиперпараметры** которого были настроены ранее по той же обучающей выборке. Т.е. learning_curve конечно проводит обучение заново, но только для параметров случайного леса, а гиперпараметры остаются прежними. По хорошему, необходимо в learning_curve передать объект grid_search и для каждого размера обучающей выборки производить поиск по сетке, но это будет слишком долго для данной учебной задачи.

## Выводы

1. Построена модель прогнозирования среднего числа посещений объявления с качеством accuracy = 0.386, что почти в два раза больше, чем прогнозирование константным значением (accuracy = 0.20)
2. Точнее всего модель предсказывает значения, которые находятся в крайних интервалах - [0.0888, 1.222] и (13.193, 324] 
3. Судя по значимости слов, люди обращают внимание на пол, возраст, медицинские и физиологические характеристики, умение гулять на поводке
4. Качество модели может быть улучшено, если к тексту добавить фотографии. Для извлечения признаков из фото можно попробовать использовать сверточные нейронные сети, например, AlexNet

## Сылки


1. [Text classification for russian language][1]
+ [Russian stemming algorithm][2]
+ [nltk.stem.snowball.RussianStemmer][3]
+ [Classification of text documents using sparse features][4]


[1]: http://stackoverflow.com/questions/18011756/text-classification-for-russian-language
[2]: http://snowball.tartarus.org/algorithms/russian/stemmer.html
[3]: http://www.nltk.org/api/nltk.stem.html
[4]: http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html